In [1]:
#Imports
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [2]:
#Search for target protein
target = new_client.target
target_query = target.search('CHEMBL243')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'Q72874', 'xref_name': None, 'xre...",Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,15.0,False,CHEMBL243,"[{'accession': 'Q72874', 'component_descriptio...",SINGLE PROTEIN,11676


Assign the eigth entry, Influenza A virus Matrix protein M2, to the ***selected_target*** variable and retrieve bioactivity data thats reported as IC<sub>50</sub> values in nM unit.

In [109]:
selected_target = targets.target_chembl_id[0]
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')
df = pd.DataFrame.from_dict(res)
df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32165,[],CHEMBL763424,Inhibitory concentration against HIV-1 Protease at pH 5.6 and at 37 degrees C,B,None,None,BAO_0000190,BAO_0000357,single protein format,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)OC(C)(C)C)C(=O)NCC(C)C,None,None,CHEMBL1126452,Bioorg. Med. Chem. Lett.,1992,None,CHEMBL9650,None,CHEMBL9650,None,False,http://www.openphacts.org/units/MicrogramPerMilliliter,5875,=,1,True,=,None,IC50,ug.mL-1,None,30.0,CHEMBL243,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,ug ml-1,UO_0000274,None,30.0
1,None,32166,[],CHEMBL763424,Inhibitory concentration against HIV-1 Protease at pH 5.6 and at 37 degrees C,B,None,None,BAO_0000190,BAO_0000357,single protein format,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)OC(C)(C)C)C(=O)NCC(C)C,Outside typical range,"Values for this activity type are unusually large/small, so may not be accurate",CHEMBL1126452,Bioorg. Med. Chem. Lett.,1992,None,CHEMBL273396,None,CHEMBL273396,None,False,http://www.openphacts.org/units/MicrogramPerMilliliter,5874,=,1,True,=,None,IC50,ug.mL-1,None,93.0,CHEMBL243,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,ug ml-1,UO_0000274,None,93.0
2,None,32456,[],CHEMBL769366,In vivo antiviral activity (IC50) against HIV-1 protease.,B,None,None,BAO_0000190,BAO_0000218,organism-based format,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc1ccc(F)cc1)NC(=O)[C@@H](NC(=O)OC)C(C)C)C(C)C,None,None,CHEMBL1129270,J. Med. Chem.,1996,"{'bei': '11.94', 'le': '0.23', 'lle': '4.87', 'sei': '4.66'}",CHEMBL108102,None,CHEMBL108102,7.38,False,http://www.openphacts.org/units/Nanomolar,205360,=,1,True,=,None,IC50,nM,None,42.0,CHEMBL243,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,nM,UO_0000065,None,42.0
3,None,32459,[],CHEMBL769366,In vivo antiviral activity (IC50) against HIV-1 protease.,B,None,None,BAO_0000190,BAO_0000218,organism-based format,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN(CC1CCCCC1)NC(=O)C(C(C)C)N(C)C(=O)OC)C(C)C,None,None,CHEMBL1129270,J. Med. Chem.,1996,None,CHEMBL322033,None,CHEMBL322033,None,False,http://www.openphacts.org/units/Nanomolar,205363,>,1,True,>,None,IC50,nM,None,1000.0,CHEMBL243,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,nM,UO_0000065,None,1000.0
4,None,33353,[],CHEMBL696055,Inhibitory activity was determined against HIV-1 protease.,B,None,None,BAO_0000190,BAO_0000357,single protein format,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc2ccccc2)cc1)C(=O)C(F)(F)C(=O)N[C@@H](C(=O)O)C(C)C,None,None,CHEMBL1127883,Bioorg. Med. Chem. Lett.,1994,"{'bei': '12.32', 'le': '0.23', 'lle': '3.73', 'sei': '5.24'}",CHEMBL20143,None,CHEMBL20143,8.40,False,http://www.openphacts.org/units/Nanomolar,25133,=,1,True,=,None,IC50,nM,None,4.0,CHEMBL243,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,nM,UO_0000065,None,4.0


## Datapreprocessing

In [114]:
#Drop missing data
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2 = df2.drop_duplicates(['canonical_smiles'])
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2[selection]
df3 = df3.dropna()

<ipython-input-114-2758f2c2ef8c>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[df.canonical_smiles.notna()]


In [115]:
#Label compounds as active, inactive or intermediate
bioactivity_threshold = []
for colname, colvals in df3.iterrows():
    if float(colvals[2]) >= 10000.0:
        bioactivity_threshold.append('inactive')
    elif float(colvals[2]) <= 1000.0:
        bioactivity_threshold.append('active')
    else:
        bioactivity_threshold.append('intermediate')

#Create dataframe of compund labels and:
#molecule_chembl_id
#canonical_smiles
#standard_value
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df3 = df3.dropna( axis=1, how='any')
df3 = df3.reset_index()
df3 = pd.concat([df3, bioactivity_class], axis=1)
df3.to_csv('bioactivity_data.csv', index=False)

In [116]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
df3

<ipython-input-116-ec088a3546e1>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,index,molecule_chembl_id,canonical_smiles,standard_value,class
0,0,CHEMBL9650,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)OC(C)(C)C)C(=O)NCC(C)C,30.0,active
1,1,CHEMBL273396,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)OC(C)(C)C)C(=O)NCC(C)C,93.0,active
2,2,CHEMBL108102,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc1ccc(F)cc1)NC(=O)[C@@H](NC(=O)OC)C(C)C)C(C)C,42.0,active
3,3,CHEMBL322033,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN(CC1CCCCC1)NC(=O)C(C(C)C)N(C)C(=O)OC)C(C)C,1000.0,active
4,4,CHEMBL20143,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc2ccccc2)cc1)C(=O)C(F)(F)C(=O)N[C@@H](C(=O)O)C(C)C,4.0,active
5,5,CHEMBL75310,Cc1ccc(CN2CCN(C[C@@H](O)[C@H](Cc3ccccc3)NC(=O)O[C@@H]3CCS(=O)(=O)[C@@H]3C(C)C)[C@H](C(=O)NC(C)(C)C)C2)cn1,2.0,active
6,6,CHEMBL19545,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc2ccccc2)cc1)C(=O)C(F)(F)C(=O)N[C@@H](C(=O)OC(C)(C)C)C(C)C,6.0,active
7,7,CHEMBL320783,COC(=O)NC(C(=O)NN(Cc1ccc(OC)cc1)C[C@H](O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)c1ccc2ccccc2n1)C(C)C,31.0,active
8,8,CHEMBL9661,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@](=O)(OC)[C@@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)OC(C)(C)C)C(=O)NCC(C)C,400.0,active
9,9,CHEMBL309346,CC(C)(C)NC(=O)[C@@H]1CN(C(=O)OCc2ccccc2)CCN1C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)O[C@H]1CCOC1,18.9,active
